# LLM baseline setup

This document outlines the steps to set up a baseline environment for working with Meta Llama-3.1. Follow the instructions below to get started:

## Prerequisites
- Google Colab.
- Hugging Face account to access Meta Llama-3.1.
- NVIDIA-A100 GPU or equivalent.
- Google Drive to store the strace logs from dynamic analysis.
- Python 3.10 or higher.

## Instructions to run
1. Open Google Colab and open this notebook.
2. Ensure that the runtime is set to use a GPU (preferably NVIDIA-A100).
3. Mount your Google Drive to access the strace logs.
4. Install the required dependencies by running the first cell in the notebook.
5. Provide the directory path to the Google Drive containing the strace log files.
6. Follow the subsequent cells to load the data, set up the LLMs, and evaluate their performance on the strace logs.
7. Extract the paths of the strace log files uploaded to the Google Drive and pass it into the `evaluate_LLM` function. Example of running provided in cells below cell 7.

In [ ]:
# Installing all the dependencies
!pip install pandas transformers torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path
import os
import pandas as pd
import re

In [ ]:
os.environ["HUGGING_FACE_HUB_TOKEN"] = "your-token-here" # Obtain the access token from hugging face

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    revision="27a2933fb533a76b333e9d1db52f3e73d27687ab"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
# Constants
N = 1000
MALICIOUS_DIR = '/content/drive/path/to/malicious/logs'
BENIGN_DIR = '/content/drive/path/to/benign/logs'
mal_files_log = list(Path(MALICIOUS_DIR).glob('*_logs.log'))
ben_files_log = list(Path(BENIGN_DIR).glob("*_logs.log"))[:N]
output_dir = os.path.join(MALICIOUS_DIR, "outputs")
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def extract_key_syscalls(strace_content):
    """Extract only the most relevant system calls for analysis"""
    key_patterns = [
        # Process execution and control
        'execve', 'fork', 'clone', 'clone3', 'vfork', 'ptrace', 'select', 'ioctl'

        # Network operations
        'socket', 'connect', 'bind', 'accept', 'listen', 'send', 'epoll_creat'
        'sendto', 'recvfrom', 'sendmsg', 'recvmsg', 'setsockopt', 'poll'

        # File operations
        'open', 'openat', 'creat', 'unlink', 'unlinkat', 'rename', 'renameat', 'getdents', 'fstat', 'fstat64'
        'write', 'read', 'access', 'faccessat', 'chmod', 'chown', 'ftruncate', 'fadvise64'

        # Memory operations
        'mmap', 'munmap', 'mmap2', 'mprotect', 'mremap', 'brk'

        # Directory operations
        'mkdir', 'mkdirat', 'rmdir', 'chdir',

        # Process control
        'kill', 'tkill', 'prctl', 'setuid', 'setgid', 'rt_sigaction', 'rt_sigprocmask',
        'tgkill', 'sched_yield'

        # time operations
        'utime'
    ]

    lines = strace_content.split('\n')
    relevant_lines = []

    for line in lines:
        if any(pattern in line for pattern in key_patterns):
            relevant_lines.append(line)

    return '\n'.join(relevant_lines[-1000:]) # Since the most relevant stuff usually happens at the last few lines

def get_strace_log(log_path, filter=False):
    with open(log_path, 'r') as f:
        if not filter:
          return f.read()[-50000:]
        return extract_key_syscalls(f.read())

def get_processed_trace_features(parquet_path):
    df = pd.read_parquet(parquet_path, engine='pyarrow')
    return df

def query_hf(sys_prompt, context, temperature=0, max_new_tokens=512):
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": str(context)}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    # Generate response
    with torch.no_grad():
      output = model.generate(
          inputs,
          temperature=temperature,
          max_new_tokens=max_new_tokens,
          pad_token_id=tokenizer.eos_token_id
      )

    torch.cuda.empty_cache()
    response = tokenizer.decode(output[0][inputs.shape[-1]:], skip_special_tokens=True)
    return response

def evaluate_LLM_raw_features(mal_files_log, query_llm, mode, filter=False):
    """
    Evaluating LLMs on raw strace logs
    """

    count_malicious, count_benign, count_failed = 0, 0, 0
    system_prompt_raw_strace = """You are analyzing strace logs to detect malicious activity in Python pickle files.

        Look for RED FLAGS like these:
        - Process creation (execve, fork, clone, ...)
        - Network connections (socket, connect, bind, ...)
        - File writes outside normal directories
        - Access to sensitive files (/etc/passwd, SSH keys)
        - Essentially Unusual system calls during deserialization

        IMPORTANT: You must respond in EXACTLY this format:
        Answer: Malicious
        Reason: Brief explanation

        OR

        Answer: Benign
        Reason: Brief explanation

        Example 1:
        Log shows: execve("/bin/bash", ["/bin/bash", "-c", "rm -rf /"])
        Answer: Malicious
        Reason: Executes destructive shell command during deserialization

        Example 2:
        Log shows: read(3, "model weights data...", 1024)
        Answer: Benign
        Reason: Normal model data reading without suspicious system calls

        Do not include any other text.
        """

    for mal_file in mal_files_log:
        print(f"Analysis has begun for {mal_file}")
        context = get_strace_log(mal_file, filter)
        response = query_llm(system_prompt_raw_strace, context)

        answer = response
        fname = os.path.basename(mal_file)
        file_path = os.path.join(output_dir, f"{mode}_{str(fname)[:-4]}_output.txt")
        with open(file_path, 'w') as f:
          if "Answer: Malicious" in answer:
              print(f"{fname} is detected as malicious")
              count_malicious += 1
          elif "Answer: Benign" in answer:
              print(f"{fname} is detected as benign")
              count_benign += 1
          else:
              print(f"Something went wrong in the LLM's response for file {fname}")
              count_failed += 1

          f.write(answer)

    print(f"Number of malicious: {count_malicious}")
    print(f"Number of benign: {count_benign}")
    print(f"Number of failed outputs: {count_failed}")

def evaluate_LLM_processed_features(df, query_llm, mode):
    """
    Evaluating on processed features from CSV/Parquet.
    """
    count_malicious, count_benign, count_failed = 0, 0, 0
    system_prompt_processed_strace = """
        You are analyzing processed strace features (frequency of system calls, sequences, and process-specific sequences) to detect malicious activity in Python pickle files.

        Look for these RED FLAGS in the feature dictionary:

        High frequency of process creation calls (execve, fork, clone)

        Presence of network-related calls (socket, connect, bind)

        Frequent or unusual file write sequences suggesting writes outside normal directories

        Access patterns indicating sensitive file reads (/etc/passwd, SSH keys, authentication files)

        Unusual system call sequences during deserialization (e.g., system calls not typical for reading model weights)

        The features can be in one of three forms:

        {system_call} -> frequency of that system call

        seq_{system_call1, system_call2} -> frequency of consecutive system calls


        proc_seq_{P1: system_call1, P2: system_call2} -> frequency of call sequences tied to specific process IDs

        IMPORTANT: You must respond in EXACTLY this format:

        Answer: Malicious
        Reason: Brief explanation


        OR

        Answer: Benign
        Reason: Brief explanation


        Example 1:
        Features show: {"execve": 3, "socket": 2}

        Answer: Malicious
        Reason: Multiple process creation and network calls detected during deserialization


        Example 2:
        Features show: {"read": 150, "seq_open,read": 50, "write": 2}

        Answer: Benign
        Reason: Mostly file reading activity consistent with loading model weights, minimal writes
        """
    for row in df:
        print(f"Analysis has begun for {row['name']}")

        response = query_llm(system_prompt_processed_strace, row)

        answer = response
        repo_name = row['name'].replace("/", "__")

        file_path = os.path.join(output_dir, f"{mode}_{repo_name}_output.txt")
        with open(file_path, 'w') as f:
          if "Answer: Malicious" in answer:
              print(f"{row['name']} is detected as malicious")
              count_malicious += 1
          elif "Answer: Benign" in answer:
              print(f"{row['name']} is detected as benign")
              count_benign += 1
          else:
              print(f"Something went wrong in the LLM's response for file {row['name']}")
              count_failed += 1

          f.write(answer)

    print(f"Number of malicious: {count_malicious}")
    print(f"Number of benign: {count_benign}")
    print(f"Number of failed outputs: {count_failed}")

def evaluate_LLM(data, mode="raw"):
  if "raw" in mode:
    print("Running raw mode")
    filter = False
    if "filter" in mode:
      filter = True
    evaluate_LLM_raw_features(data, query_hf, mode, filter=filter)
  elif "processed" in mode:
    print("Running processed mode")
    evaluate_LLM_processed_features(data, query_hf, mode)
    print("Analysis has ended")

In [ ]:
MALICIOUS_DIR = '/content/drive/MyDrive/MALHUG_logs/MALHUG_injected'
mal_files_log = list(Path(MALICIOUS_DIR).glob('*_logs.log'))
output_dir = os.path.join(MALICIOUS_DIR, "outputs")
os.makedirs(output_dir, exist_ok=True)
evaluate_LLM(mal_files_log, "raw_filtered")

Running raw mode
Analysis has begun for /content/drive/MyDrive/MALHUG_logs/MALHUG_injected/strace_logs_groar__gpt-neo-1.3B-finetuned-escape3--groar__gpt-neo-1.3B-finetuned-escape3__pytorch_model.bin_logs.log
strace_logs_groar__gpt-neo-1.3B-finetuned-escape3--groar__gpt-neo-1.3B-finetuned-escape3__pytorch_model.bin_logs.log is detected as malicious
Analysis has begun for /content/drive/MyDrive/MALHUG_logs/MALHUG_injected/strace_logs_huggingtweets___f1rewalker_-staticmeganito--huggingtweets___f1rewalker_-staticmeganito__pytorch_model.bin_logs.log
strace_logs_huggingtweets___f1rewalker_-staticmeganito--huggingtweets___f1rewalker_-staticmeganito__pytorch_model.bin_logs.log is detected as malicious
Analysis has begun for /content/drive/MyDrive/MALHUG_logs/MALHUG_injected/strace_logs_NoLawz__DialoGPT-medium-harrypotter--NoLawz__DialoGPT-medium-harrypotter__pytorch_model.bin_logs.log
strace_logs_NoLawz__DialoGPT-medium-harrypotter--NoLawz__DialoGPT-medium-harrypotter__pytorch_model.bin_logs.l

Examples to running the evaluation functions are provided in the cells below.

In [ ]:
MALICIOUS_DIR = '/content/drive/MyDrive/MALHUG_logs/MALHUG_tg/text-generation'
mal_files_log = list(Path(MALICIOUS_DIR).glob('*_logs.log'))
output_dir = os.path.join(MALICIOUS_DIR, "outputs")
os.makedirs(output_dir, exist_ok=True)
evaluate_LLM(mal_files_log, "raw_filtered")


In [ ]:
MALICIOUS_DIR = '/content/drive/MyDrive/MALHUG_logs/PyPI/pypi_injected'
mal_files_log = list(Path(MALICIOUS_DIR).glob('*_logs.log'))
output_dir = os.path.join(MALICIOUS_DIR, "outputs")
os.makedirs(output_dir, exist_ok=True)
evaluate_LLM(mal_files_log, "raw_filtered")

Running raw mode
Analysis has begun for /content/drive/MyDrive/MALHUG_logs/PyPI/pypi_injected/strace_logs_huggingtweets__bognamk--huggingtweets__bognamk__pytorch_model.bin_logs.log
strace_logs_huggingtweets__bognamk--huggingtweets__bognamk__pytorch_model.bin_logs.log is detected as malicious
Analysis has begun for /content/drive/MyDrive/MALHUG_logs/PyPI/pypi_injected/strace_logs_huggingtweets__angiejolielive--huggingtweets__angiejolielive__pytorch_model.bin_logs.log
strace_logs_huggingtweets__angiejolielive--huggingtweets__angiejolielive__pytorch_model.bin_logs.log is detected as malicious
Analysis has begun for /content/drive/MyDrive/MALHUG_logs/PyPI/pypi_injected/strace_logs_huggingartists__miyagi--huggingartists__miyagi__pytorch_model.bin_logs.log
strace_logs_huggingartists__miyagi--huggingartists__miyagi__pytorch_model.bin_logs.log is detected as malicious
Analysis has begun for /content/drive/MyDrive/MALHUG_logs/PyPI/pypi_injected/strace_logs_donggyu__mnmt--donggyu__mnmt__pytorch_

In [ ]:
BENIGN_DIR = '/content/drive/MyDrive/Wild_Benign_Set/test_set'
ben_files_log = list(Path(BENIGN_DIR).glob("*_logs.log"))
output_dir = os.path.join(BENIGN_DIR, "outputs")
os.makedirs(output_dir, exist_ok=True)
evaluate_LLM(ben_files_log, "raw_filtered")


Running raw mode
Analysis has begun for /content/drive/MyDrive/Wild_Benign_Set/test_set/strace_logs_huggingtweets__hoshirisu--huggingtweets__hoshirisu__pytorch_model.bin_logs.log


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


strace_logs_huggingtweets__hoshirisu--huggingtweets__hoshirisu__pytorch_model.bin_logs.log is detected as benign
Analysis has begun for /content/drive/MyDrive/Wild_Benign_Set/test_set/strace_logs_huggingtweets__amphydelic--huggingtweets__amphydelic__pytorch_model.bin_logs.log
strace_logs_huggingtweets__amphydelic--huggingtweets__amphydelic__pytorch_model.bin_logs.log is detected as malicious
Analysis has begun for /content/drive/MyDrive/Wild_Benign_Set/test_set/strace_logs_huggingartists__joji--huggingartists__joji__pytorch_model.bin_logs.log
strace_logs_huggingartists__joji--huggingartists__joji__pytorch_model.bin_logs.log is detected as malicious
Analysis has begun for /content/drive/MyDrive/Wild_Benign_Set/test_set/strace_logs_huggingtweets__gimoyin--huggingtweets__gimoyin__pytorch_model.bin_logs.log
strace_logs_huggingtweets__gimoyin--huggingtweets__gimoyin__pytorch_model.bin_logs.log is detected as benign
Analysis has begun for /content/drive/MyDrive/Wild_Benign_Set/test_set/strac

In [ ]:
BENIGN_DIR = '/content/drive/MyDrive/Wild_Benign_Set/last1k_ben_set'
ben_files_log = list(Path(BENIGN_DIR).glob('*_logs.log'))
output_dir = os.path.join(BENIGN_DIR, "outputs")
os.makedirs(output_dir, exist_ok=True)
evaluate_LLM(ben_files_log, "raw_filtered")

In [ ]:
BENIGN_DIR = '/content/drive/MyDrive/Wild_Benign_Set/last1k_ben_set'
ben_files_log = list(Path(BENIGN_DIR).glob('*_logs.log'))
output_dir = os.path.join(BENIGN_DIR, "outputs")
os.makedirs(output_dir, exist_ok=True)
evaluate_LLM(ben_files_log,"raw_filtered")

Running raw mode
Analysis has begun for /content/drive/MyDrive/Wild_Benign_Set/last1k_ben_set/strace_logs_RyotaroOKabe__ope_dgpt2_v1.1--RyotaroOKabe__ope_dgpt2_v1.1__pytorch_model.bin_logs.log


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


strace_logs_RyotaroOKabe__ope_dgpt2_v1.1--RyotaroOKabe__ope_dgpt2_v1.1__pytorch_model.bin_logs.log is detected as benign
Analysis has begun for /content/drive/MyDrive/Wild_Benign_Set/last1k_ben_set/strace_logs_huggingartists__tiamat--huggingartists__tiamat__pytorch_model.bin_logs.log
strace_logs_huggingartists__tiamat--huggingartists__tiamat__pytorch_model.bin_logs.log is detected as malicious
Analysis has begun for /content/drive/MyDrive/Wild_Benign_Set/last1k_ben_set/strace_logs_LelouchH__DiabloGPT-small-RaidenBot--LelouchH__DiabloGPT-small-RaidenBot__pytorch_model.bin_logs.log
strace_logs_LelouchH__DiabloGPT-small-RaidenBot--LelouchH__DiabloGPT-small-RaidenBot__pytorch_model.bin_logs.log is detected as malicious
Analysis has begun for /content/drive/MyDrive/Wild_Benign_Set/last1k_ben_set/strace_logs_jtatman__gpt2-open-instruct-v1-gsm8k--jtatman__gpt2-open-instruct-v1-gsm8k__pytorch_model.bin_logs.log
strace_logs_jtatman__gpt2-open-instruct-v1-gsm8k--jtatman__gpt2-open-instruct-v1-gs

In [ ]:
from pathlib import Path
import pandas as pd

mode = "raw_filtered"

files = [f for f in Path(output_dir).glob("*.txt") if os.path.basename(f).startswith(f'{mode}')]
record = pd.DataFrame(columns=["Model_id", "Detection", "Reason"])
for file in files:
    fname = os.path.basename(file)
    pattern = fr"{mode}_strace_logs_(.*?)_logs_output\.txt"
    match = re.search(pattern, fname)

    repo_name = None
    if match:
        repo_name = match.group(1)

    with open(file, 'r') as f:
        detection = f.readline().strip().split("Answer: ")[1]
        reason = f.readline().strip().split("Reason: ")[1]

        record.loc[len(record)] = {
            "Model_id": repo_name,
            "Detection": detection,
            "Reason": reason
        }

record.to_csv(os.path.join(os.path.dirname(output_dir), f"{mode}_wild_benign_LLM-Eval.csv"), index=False)